# **Import Libraries**

In [2]:
using NamedTrajectories
using QuantumCollocation

# **Unitary Smooth Problem**

Construct a `QuantumControlProblem` for a free-time unitary gate problem with smooth control pulses enforced by constraining the second derivative of the pulse trajectory, i.e.,

\begin{aligned}
\underset{\vec{\tilde{U}}, \, a, \, \dot{a}, \, \ddot{a}, \, \Delta t}{\text{minimize}} \quad
{Q} \cdot \ell (\vec{\tilde{U}}_T, \vec{\tilde{U}}_{\text{goal}}) + \frac{1}{2} \sum_t (R_a a_t^2 + R_{\dot{a}} \dot{a}_t^2 + R_{\ddot{a}} \ddot{a}_t^2) \\ \quad 
\text{ subject to } \quad  \\
\quad {P}^{(n)}(\vec{\tilde{U}}_{t+1}, \vec{\tilde{U}}_t, a_t, \Delta t_t) = 0 \\
a_{t+1} - a_t - \dot{a}_t \Delta t_t = 0 \\
\quad \dot{a}_{t+1} - \dot{a}_t - \ddot{a}_t \Delta t_t = 0 \\
\quad |a_t| \leq a_{\text{bound}} \\
\quad |\ddot{a}_t| \leq \ddot{a}_{\text{bound}} \\
\quad \Delta t_{\text{min}} \leq \Delta t_t \leq \Delta t_{\text{max}} \\
\end{aligned}


where, for $U \in SU(N)$,

\begin{equation}
\ell (\vec{\tilde{U}}_T, \vec{\tilde{U}}_{\text{goal}}) =
{1 - \frac{1}{N}{ \text{tr} (U_{\text{goal}}, U_T)} }
\end{equation}

is the *infidelity* objective function, $Q$ is a weight, $R_a$, $R_{\dot{a}}$, and $R_{\ddot{a}}$ are weights on the regularization terms, and ${P}^{(n)}$ is the $n$th-order Pade integrator.

# **Arguments**

- `H_drift::AbstractMatrix{<:Number}`: the drift hamiltonian
- `H_drives::Vector{<:AbstractMatrix{<:Number}}`: the control hamiltonians
or
- `system::QuantumSystem`: the system to be controlled
with
- `operator::Union{EmbeddedOperator, AbstractMatrix{<:Number}}`: the target unitary, either in the form of an `EmbeddedOperator` or a `Matrix{ComplexF64}
- `T::Int`: the number of timesteps
- `Δt::Float64`: the (initial) time step size

# **Keyword Arguments**
- `free_time::Bool=true`: whether or not to allow the time steps to vary
- `init_trajectory::Union{NamedTrajectory, Nothing}=nothing`: an initial trajectory to use
- `a_bound::Float64=1.0`: the bound on the control pulse
- `a_bounds::Vector{Float64}=fill(a_bound, length(system.G_drives))`: the bounds on the control pulses, one for each drive
- `a_guess::Union{Matrix{Float64}, Nothing}=nothing`: an initial guess for the control pulses
- `dda_bound::Float64=1.0`: the bound on the control pulse derivative
- `dda_bounds::Vector{Float64}=fill(dda_bound, length(system.G_drives))`: the bounds on the control pulse derivatives, one for each drive
- `Δt_min::Float64=0.5 * Δt`: the minimum time step size
- `Δt_max::Float64=1.5 * Δt`: the maximum time step size
- `drive_derivative_σ::Float64=0.01`: the standard deviation of the initial guess for the control pulse derivatives
- `Q::Float64=100.0`: the weight on the infidelity objective
- `R=1e-2`: the weight on the regularization terms
- `R_a::Union{Float64, Vector{Float64}}=R`: the weight on the regularization term for the control pulses
- `R_da::Union{Float64, Vector{Float64}}=R`: the weight on the regularization term for the control pulse derivatives
- `R_dda::Union{Float64, Vector{Float64}}=R`: the weight on the regularization term for the control pulse second derivatives
- `leakage_suppression::Bool=false`: whether or not to suppress leakage to higher energy states
- `R_leakage=1e-1`: the weight on the leakage suppression term
- `max_iter::Int=1000`: the maximum number of iterations for the solver
- `linear_solver::String="mumps"`: the linear solver to use
- `ipopt_options::Options=Options()`: the options for the Ipopt solver
- `constraints::Vector{<:AbstractConstraint}=AbstractConstraint[]`: additional constraints to add to the problem
- `timesteps_all_equal::Bool=true`: whether or not to enforce that all time steps are equal
- `verbose::Bool=false`: whether or not to print constructor output
- `integrator=:pade`: the integrator to use for the unitary, either `:pade` or `:exponential`
- `rollout_integrator=exp`: the integrator to use for the rollout
- `bound_unitary=integrator == :exponential`: whether or not to bound the unitary
- `control_norm_constraint=false`: whether or not to enforce a constraint on the control pulse norm
- `control_norm_constraint_components=nothing`: the components of the control pulse to use for the norm constraint
- `control_norm_R=nothing`: the weight on the control pulse norm constraint
- `geodesic=true`: whether or not to use the geodesic as the initial guess for the unitary
- `pade_order=4`: the order of the Pade approximation to use for the unitary integrator
- `autodiff=pade_order != 4`: whether or not to use automatic differentiation for the unitary integrator
- `subspace=nothing`: the subspace to use for the unitary integrator
- `jacobian_structure=true`: whether or not to use the jacobian structure
- `hessian_approximation=false`: whether or not to use L-BFGS hessian approximation in Ipopt
- `blas_multithreading=true`: whether or not to use multithreading in BLAS

## **Sample Example From Documentation for Reference**

 To set up and solve a quantum optimal control problems we provide high level problem templates to quickly get started. For unitary gate problems, where we want to realize a gate $U_{\text{goal}}$, with a system Hamiltonian of the form,

$$H(t) = H_0 + \sum_{i} a^i(t) H_i$$

 there is the `UnitarySmoothPulseProblem` constructor which only requires
 - the drift Hamiltonian, $H_0$
 - the drive Hamiltonians, ${H_i}$
 - the target unitary, $U_{\text{goal}}$
 - the number of timesteps, $T$
 - the (initial) time step size, $\Delta t$

 ## **Basic Usage**

 For example, to create a problem for a single qubit $X$ gate (with a bound on the drive of $|a^i| < a_{\text{bound}}$), i.e., with system hamiltonian

$$ H(t) = \frac{\omega}{2} \sigma_z + a^1(t) \sigma_x + a^2(t) \sigma_y $$

 we can do the following:

In [2]:
# set time parameters
T = 100
Δt = 0.1

# use the exported gate dictionary to get the gates we need
σx = get_gate(:X)
σy = get_gate(:Y)
σz = get_gate(:Z)

# define drift and drive Hamiltonians
H_drift = 0.5 * σz
H_drives = [σx, σy]

# define target unitary
U_goal = σx

# set bound on the drive
a_bound = 1.0

# build the problem
prob = UnitarySmoothPulseProblem(
    H_drift,
    H_drives,
    U_goal,
    T,
    Δt;
    a_bound=a_bound,
)

# solve the problem
solve!(prob; max_iter=30)

applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:    10442
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     7428

Total number of variables............................:     1488
           

# **Sample-Based Learning (SLC) Robust Control**

To be specific, let's take a simple system:

\begin{equation}
H(a; \zeta) = \zeta Z + a_1(t) X + a_2(t) Y
\end{equation}

This is a qubit with some random dephasing noise. The noise might be due to fluctuations in the qubit frequency that persist longer than a gate but change over the timescale of multiple gates. First, we will focus on setting up and solving the control problem. Next, we can think about generalizing the QuantumSystem constructor.


### 1. **Define an ensemble of QuantumSystem objects.**

Sample to create

\begin{equation}
H(a; \zeta_i) = \zeta_i Z + a_1(t) X + a_2(t) Y
\end{equation}
for some distribution of $\zeta_i \in \mathcal{N}(0, 0.01j)$ .

From here, we could set up an independent control problem for each of these systems. Let's say we want an X gate; set T= 50 and Δt = 0.2.

###  2. **Redo UnitarySmoothPulseProblem with the help of for loops**

What we want to do now is create a `joint smooth pulse problem` that adds a unitary for each of our quantum systems, but when it comes time to add the controls, we only add a `single, shared control`! We can do this by manually `recreating the NamedTrajectory container`, basically following all the steps in a `UnitarySmoothPulseProblem`.

Check out how `add_suffix` can be used to attach a unique label to keys. We use this a lot in our `UnitaryDirectSumProblem`. We just need to create `one trajectory key` for each system. They can all even use the same kind of initial data that we supply in the `UnitarySmoothPulseProblem`.

###  3. **Glue the `trajectory` together with the right dynamics**

Now comes the interesting part! We can attach `integrators` for each system, separately! In `UnitarySmoothPulseProblem`, we have this block of code:
```
    if integrator == :pade
        unitary_integrator =
            UnitaryPadeIntegrator(system, :Ũ⃗, :a; order=pade_order, autodiff=autodiff)
    elseif integrator == :exponential
        unitary_integrator =
            UnitaryExponentialIntegrator(system, :Ũ⃗, :a)
    else
        error("integrator must be one of (:pade, :exponential)")
    end
```
Notice that we get to supply the keys. With diligent use of `for loop` and `add_suffix`, we can make the `integrators` we need for each quantum system in our ensemble.

###  4. **Make sure we have all the pieces**!

Check out the final return statement:
```
return QuantumControlProblem(
        system,
        traj,
        J,
        integrators;
        constraints=constraints,
        max_iter=max_iter,
        linear_solver=linear_solver,
        verbose=verbose,
        ipopt_options=ipopt_options,
        jacobian_structure=jacobian_structure,
        hessian_approximation=hessian_approximation,
        eval_hessian=!hessian_approximation,
        kwargs...
    )
```
Make sure you collect all the pieces you need. For example, we pass our fancy, joint trajectory and list of many integrators. We also want the quantum system to be the `direct_sum` of all the supplied systems!


# **Pedagogical Details about Sample-based Robust Control**

The [paper](https://arxiv.org/pdf/1507.07190) outlines a systematic methodology for sampling-based learning control (SLC) in quantum systems with uncertainties. It involves two steps: "training" and "testing." In training, an augmented system is created using artificial samples generated from uncertainty parameters, and a learning algorithm determines the control. In testing, additional samples are evaluated for control performance. The [paper](https://arxiv.org/pdf/1507.07190) demonstrates the method's efficacy through three quantum control examples: state preparation in a three-level quantum system, robust entanglement generation in a two-qubit superconducting circuit, and quantum entanglement control in a two-atom system interacting with a quantized field in a cavity.

In the training step, $(N)$ samples are first generated by sampling the uncertainty parameters according to a given probability distribution (e.g., the uniform distribution or Normal Distribution). The combination $(\theta_{0n_0}, \theta_{1n_1}, \ldots, \theta_{Mn_M})$ is chosen from the sampled parameters $\{\theta_{jn_j}, j=0,1,\ldots,M\}$, where $\theta_{jn_j}$ is a possible value of the uncertainty parameter $\theta_j, n_j=1, \ldots,N_j$, and $N_j$ is the number of samples of the parameter $\theta_j (j=0,1,\ldots,M)$. The total number of potential samples is given by $N = \prod_{j=0}^M N_j$. These different sample systems are denoted as $\{\mathbf{n} = (n_1, \ldots, n_N)\}$ and $\Theta_{\mathbf{n}} \in \{(\theta_{0n_0}, \theta_{1n_1}, \ldots, \theta_{Mn_M})|n_j=1, \ldots, N_j\}$. 

Using these samples, an augmented system can be constructed as follows:

$$
\begin{equation}
\begin{aligned}
\frac{d}{dt}
\begin{pmatrix}
|\psi_1(t)\rangle \\
|\psi_2(t)\rangle \\
\vdots \\
|\psi_N(t)\rangle 
\end{pmatrix}
= -\frac{i}{\hbar}
\begin{pmatrix}
H_{\Theta_1}(t)|\psi_1(t)\rangle \\
H_{\Theta_2}(t)|\psi_2(t)\rangle \\
\vdots \\
H_{\Theta_N}(t)|\psi_N(t)\rangle 
\end{pmatrix}
\end{aligned}
\end{equation}$$

where $H_{\Theta_n}(t) \equiv f_0(\theta_{0n_0})H_0 + \sum_{m} f_m(\theta_{mn_m}) u_m(t)H_m$ with $n=1,2,\ldots,N$. The performance function for the augmented system is defined as follows:



\begin{equation}
\begin{aligned}
J_N(u) &:= \frac{1}{N} \sum_{n=1}^N J_{\Theta_n}(u) \\
&= \frac{1}{N} \sum_{n=1}^N \left| \langle \psi_n(T) | \psi_{\text{target}} \rangle \right|^2.
\end{aligned}
\end{equation}


The task of the training step is to find a control strategy \(u^*\) that maximizes the performance function defined in Eq. (2). In an ideal scenario, there are no uncertainties in (3).

\begin{equation}
\begin{aligned}
H(t) = H_0 + H_c(t) = H_0 + \sum_{m=1}^{M} u_m(t) H_m.
\end{aligned}
\end{equation}


However, in practical quantum systems, uncertainties are inevitable due to external disturbances, imperfect models, and inaccuracies in control fields. The Hamiltonian of the system is given by:


\begin{equation}
\begin{aligned}
H_{\Theta}(t) = f_0(\theta_0)H_0 + \sum_{m=1}^M f_m(\theta_m)u_m(t)H_m.
\end{aligned}
\end{equation}


Here, the parameters are represented by $\Theta = (\theta_0, \theta_1, \ldots, \theta_M)$, and the functions $f_j(\theta_j) (j = 0, 1, \ldots, M)$ describe potential uncertainties. For instance, $f_0(\theta_0)$ might indicate uncertainties in the free Hamiltonian (e.g., due to chemical shifts in NMR). The function $f_m(\theta_m)$ could represent multiplicative noise in control fields or inaccuracies in the dipole approximation. If $f_m(\theta_m)$ are time-varying, the associated uncertainties may arise from time-dependent errors in the control fields.

Additive noises in control fields can also be included by modifying equation (4) slightly. Assume $f_j(\theta_j)$ are continuous functions of $\theta_j$ and these parameters may vary with time, with $\theta_j$ in the range $[1 - E_j, 1 + E_j]$. For simplicity, assume all uncertainty bounds $E_0 = \ldots = E_j = \ldots = E_M = E$ are equal. Suppose the nominal values of $\theta_j$ are 1, with fluctuations within $(2E) (E \in [0,1])$. The goal is to design control functions $u_m(t)) for (m = 1, 2, \ldots, M$ that guide the quantum system from its initial state $(|\psi_0\rangle)$ to a target state $(|\psi_{\text{target}}\rangle)$ with high fidelity.

Choosing an appropriate number of samples $( N)$ is essential for accurately representing the distributions of the uncertainty parameters $( \theta_j \in [1 - E, 1 + E])$. The primary motivation for the proposed sampling-based approach is to design the control law using artificial samples in place of unknown uncertainties. Therefore, it is crucial to select a representative set of samples for these uncertainty parameters.

Consider the scenario with two uncertainty parameters $(\theta_0)$ and $(\theta_1 )$. When the distributions of both $( \theta_0)$ and $( \theta_1)$ are uniform, it is possible to choose equally spaced samples for $(\theta_0)$ and $(\theta_1 )$. For instance, the intervals $([1 - E, 1 + E])$ for $(\theta_0)$ and $([1 - E, 1 + E])$ for $(\theta_1)$ can be divided into $(N_0 + 1)$ and $(N_1 + 1)$ subintervals, respectively, where $ (N_0)$ and $( N_1)$ are typically positive odd numbers. Consequently, the total number of samples is $( N = N_0 N_1)$, and $( \Theta_n = (\theta_{0n_0}, \theta_{1n_1}))$ is selected from the set of sample points

$$
\begin{aligned}
\Theta_n \in \{ (\theta_{0n_0}, \theta_{1n_1}): 
\begin{cases}
\theta_{0n_0} = 1 - E + \frac{(2n_0 - 1)E}{N_0}, \\
\theta_{1n_1} = 1 + E + \frac{(2n_1 - 1)E}{N_1}, \\
n_0 \in \{1, \ldots, N_0\}, \ n_1 \in \{1, \ldots, N_1\}.
\end{cases}
\end{aligned}
$$

In practical applications, the values of $( N_0)$ and $( N_1)$ can be determined based on experience or numerical computation. As long as the augmented system effectively models the quantum system with uncertainties and facilitates the identification of the optimal control strategy, it is preferable to select smaller values for $( N_0)$ and $( N_1 )$ to expedite the training process and simplify the augmented system. Numerical results indicate that utilizing seven samples for each uncertainty parameter yields excellent performance.



## **Creating Ensemble of Quantum Systems**

In [3]:
using Distributions
using LinearAlgebra
# 1. Define an ensemble of QuantumSystem objects
N = 6
ζ_dist = Normal(0, 0.01)
ζ_samples = rand(ζ_dist, N)

# use the exported gate dictionary to get the gates we need
σx = get_gate(:X)
σy = get_gate(:Y)
σz = get_gate(:Z)

# Define the Hamiltonians
H_drift = [ζ * σz for ζ in ζ_samples]
H_drives = [σx, σy]

# Create QuantumSystem objects
quantum_systems = [QuantumSystem(H_drift[i], H_drives) for i in 1:N]

6-element Vector{QuantumSystem}:
 QuantumSystem(sparse([1, 2], [1, 2], ComplexF64[0.0034099385969962076 + 0.0im, -0.0034099385969962076 + 0.0im], 2, 2), SparseArrays.SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1], [1, 2], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im], 2, 2), sparse([2, 1], [1, 2], ComplexF64[0.0 + 1.0im, 0.0 - 1.0im], 2, 2)], sparse([3, 4, 1, 2], [1, 2, 3, 4], [-0.0034099385969962076, 0.0034099385969962076, 0.0034099385969962076, -0.0034099385969962076], 4, 4), SparseArrays.SparseMatrixCSC{Float64, Int64}[sparse([4, 3, 2, 1], [1, 2, 3, 4], [-1.0, -1.0, 1.0, 1.0], 4, 4), sparse([2, 1, 4, 3], [1, 2, 3, 4], [1.0, -1.0, 1.0, -1.0], 4, 4)], 2, nothing, Dict{Symbol, Any}())
 QuantumSystem(sparse([1, 2], [1, 2], ComplexF64[-0.002963447189926772 - 0.0im, 0.002963447189926772 - 0.0im], 2, 2), SparseArrays.SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1], [1, 2], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im], 2, 2), sparse([2, 1], [1, 2], ComplexF64[0.0 + 1.0im, 0.0 - 1.0im], 2, 2)], sparse([3,

# **Draft: In Progresss for SampleBasedControlProblem**

In [24]:
function SampleBasedControlProblem(
    quantum_systems::AbstractVector{<:QuantumSystem}, # Ensemble of Quantum Systems
    operator::Union{EmbeddedOperator, AbstractMatrix{<:Number}},
    T::Int,
    Δt::Union{Float64, Vector{Float64}};
    free_time=true,
    init_trajectory::Union{NamedTrajectory, Nothing}=nothing,
    a_bound::Float64=1.0,
    a_bounds=fill(a_bound, length(quantum_systems[1].G_drives)),
    a_guess::Union{Matrix{Float64}, Nothing}=nothing,
    dda_bound::Float64=1.0,
    dda_bounds=fill(dda_bound, length(quantum_systems[1].G_drives)),
    Δt_min::Float64=0.5 * Δt,
    Δt_max::Float64=1.5 * Δt,
    drive_derivative_σ::Float64=0.01,
    Q::Float64=100.0,
    R=1e-2,
    R_a::Union{Float64, Vector{Float64}}=R,
    R_da::Union{Float64, Vector{Float64}}=R,
    R_dda::Union{Float64, Vector{Float64}}=R,
    leakage_suppression=false,
    R_leakage=1e-1,
    max_iter::Int=1000,
    linear_solver::String="mumps",
    ipopt_options::Options=Options(),
    constraints::Vector{<:AbstractConstraint}=AbstractConstraint[],
    timesteps_all_equal::Bool=true,
    verbose::Bool=false,
    integrator::Symbol=:pade,
    rollout_integrator=exp,
    bound_unitary=integrator == :exponential,
    control_norm_constraint=false,
    control_norm_constraint_components=nothing,
    control_norm_R=nothing,
    geodesic=true,
    pade_order=4,
    autodiff=pade_order != 4,
    jacobian_structure=true,
    hessian_approximation=false,
    blas_multithreading=true,
    kwargs...
)
    if operator isa EmbeddedOperator
        U_goal = operator.operator
        U_init = get_subspace_identity(operator)
    else
        U_goal = Matrix{ComplexF64}(operator)
        U_init = Matrix{ComplexF64}(I(size(U_goal, 1)))
    end

    if !blas_multithreading
        BLAS.set_num_threads(1)
    end

    if hessian_approximation
        ipopt_options.hessian_approximation = "limited-memory"
    end

    n_drives = length(quantum_systems[1].G_drives)
    prob_labels = [string(i) for i ∈ 1:length(quantum_systems)]
    initial_trajectories = []

    for (i, system) in enumerate(quantum_systems)
        suffix = prob_labels[i]

        if !isnothing(init_trajectory)
            traj = add_suffix(init_trajectory, suffix)
        else
            if free_time
                if Δt isa Float64
                    Δt = fill(Δt, 1, T)
                end
            end

            if isnothing(a_guess)
                if geodesic
                    if operator isa EmbeddedOperator
                        Ũ⃗ = unitary_geodesic(operator, T)
                    else
                        Ũ⃗ = unitary_geodesic(U_goal, T)
                    end
                else
                    Ũ⃗ = unitary_linear_interpolation(U_init, U_goal, T)
                end

                if a_bounds isa AbstractVector
                    a_dists = [Uniform(-a_bounds[i], a_bounds[i]) for i = 1:n_drives]
                elseif a_bounds isa Tuple
                    a_dists = [Uniform(aᵢ_lb, aᵢ_ub) for (aᵢ_lb, aᵢ_ub) ∈ zip(a_bounds...)]
                else
                    error("a_bounds must be a Vector or Tuple")
                end

                a = hcat([
                    zeros(n_drives),
                    vcat([rand(a_dists[i], 1, T - 2) for i = 1:n_drives]...),
                    zeros(n_drives)
                ]...)

                da = randn(n_drives, T) * drive_derivative_σ
                dda = randn(n_drives, T) * drive_derivative_σ
            else
                Ũ⃗ = unitary_rollout(
                    operator_to_iso_vec(U_init),
                    a_guess,
                    Δt,
                    system;
                    integrator=rollout_integrator
                )
                a = a_guess
                da = derivative(a, Δt)
                dda = derivative(da, Δt)

                # to avoid constraint violation error at initial iteration
                da[:, end] = da[:, end-1] + Δt[end-1] * dda[:, end-1]
            end

            initial = (
                Ũ⃗ = operator_to_iso_vec(U_init),
                a = zeros(n_drives),
            )

            final = (
                a = zeros(n_drives),
            )

            goal = (
                Ũ⃗ = operator_to_iso_vec(U_goal),
            )

            bounds = (
                a = a_bounds,
                dda = dda_bounds,
            )

            if bound_unitary
                Ũ⃗_dim = size(Ũ⃗, 1)
                Ũ⃗_bound = (
                    Ũ⃗ = (-ones(Ũ⃗_dim), ones(Ũ⃗_dim)),
                )
                bounds = merge(bounds, Ũ⃗_bound)
            end

            if free_time
                components = (
                    Ũ⃗ = Ũ⃗,
                    a = a,
                    da = da,
                    dda = dda,
                    Δt = Δt,
                )

                bounds = merge(bounds, (Δt = (Δt_min, Δt_max),))

                traj = NamedTrajectory(
                    components;
                    controls=(:dda, :Δt),
                    timestep=:Δt,
                    bounds=bounds,
                    initial=initial,
                    final=final,
                    goal=goal
                )
            else
                components = (
                    Ũ⃗ = Ũ⃗,
                    a = a,
                    da = da,
                    dda = dda,
                )

                traj = NamedTrajectory(
                    components;
                    controls=(:dda,),
                    timestep=Δt,
                    bounds=bounds,
                    initial=initial,
                    final=final,
                    goal=goal
                )
            end
        end

        push!(initial_trajectories, traj)
    end
   
   println(initial_trajectories)
   println(traj)
    
   #step 2: working on traj. , Having problem with combining trajectories!
   traj = direct_sum([add_suffix(traj, ℓ) for (traj, ℓ) ∈ zip(initial_trajectories, prob_labels)])
    
   # Step 3: Setup integrators for each quantum system, Hope this is okay!
    integrators = []
    for (i, system) in enumerate(quantum_systems)
        suffix = prob_labels[i]
        if integrator == :pade
            unitary_integrator = UnitaryPadeIntegrator(system, :Ũ⃗, :a; order=pade_order, autodiff=autodiff)
        elseif integrator == :exponential
            unitary_integrator = UnitaryExponentialIntegrator(system, :Ũ⃗, :a)
        else
            error("integrator must be one of (:pade, :exponential)")
        end
        push!(integrators, add_suffix(unitary_integrator, suffix))
    end

    integrators = vcat(
        integrators,
        DerivativeIntegrator(:a, :da, traj),
        DerivativeIntegrator(:da, :dda, traj)
    )
    
    # Define the Objective Function
    J = UnitaryInfidelityObjective(:Ũ⃗, traj, Q;
        subspace=operator isa EmbeddedOperator ? operator.subspace_indices : nothing,
    )
    J += QuadraticRegularizer(:a, traj, R_a)
    J += QuadraticRegularizer(:da, traj, R_da)
    J += QuadraticRegularizer(:dda, traj, R_dda)

    if leakage_suppression
        if operator isa EmbeddedOperator
            leakage_indices = get_unitary_isomorphism_leakage_indices(operator)
            J_leakage, slack_con = L1Regularizer(
                :Ũ⃗,
                traj;
                R_value=R_leakage,
                indices=leakage_indices
            )
            push!(constraints, slack_con)
            J += J_leakage
        else
            @warn "leakage_suppression is not supported for non-embedded operators, ignoring."
        end
    end

    if free_time
        if timesteps_all_equal
            push!(constraints, TimeStepsAllEqualConstraint(:Δt, traj))
        end
    end

    if control_norm_constraint
        @assert !isnothing(control_norm_constraint_components) "control_norm_constraint_components must be provided"
        @assert !isnothing(control_norm_R) "control_norm_R must be provided"
        norm_con = ComplexModulusContraint(
            :a,
            control_norm_R,
            traj;
            name_comps=control_norm_constraint_components,
        )
        push!(constraints, norm_con)
    end

    return QuantumControlProblem(
        system,
        traj,
        J,
        integrators;
        constraints=constraints,
        max_iter=max_iter,
        linear_solver=linear_solver,
        verbose=verbose,
        ipopt_options=ipopt_options,
        jacobian_structure=jacobian_structure,
        hessian_approximation=hessian_approximation,
        eval_hessian=!hessian_approximation,
        kwargs...
    )
end


SampleBasedControlProblem (generic function with 1 method)

## **Basic Usage of SLC**

In [26]:
# set time parameters
T = 50;
Δt = 0.2

# use the exported gate dictionary to get the gates we need
σx = get_gate(:X)
σy = get_gate(:Y)
σz = get_gate(:Z)

# Target unitary
U_goal = σx

# Set bound on the drive
a_bound = 1.0

# Define options for the optimizer
options = Options(max_iter=30)


# Create SampleBasedControlProblem
prob = SampleBasedControlProblem(
    quantum_systems,
    U_goal,
    T,
    Δt;
    a_bound=a_bound,
)

solve!(prob; options) # :(

Any[NamedTrajectory{Float64}([1.0 0.9989726963751682 0.9958950069116229 0.9907795784955328 0.9836474315195147 0.9745278735053341 0.963458378673011 0.9504844339512096 0.9356593520616949 0.9190440524459204 0.9007068109339783 0.8807229791845672 0.8591746750488638 0.8361504451306584 0.811744900929367 0.7860583300610849 0.7591962841552627 0.7312691451204175 0.7023916715611972 0.672682527210654 0.6422637933155163 0.6112604669781573 0.5797999475166895 0.5480115129538409 0.5160257887858277 0.48397421121417267 0.45198848704615924 0.4202000524833106 0.388739533021843 0.35773620668448386 0.3273174727893465 0.29760832843880314 0.26873085487958265 0.24080371584473764 0.21394166993891528 0.18825509907063334 0.1638495548693418 0.14082532495113648 0.11927702081543291 0.09929318906602186 0.08095594755407974 0.06434064793830535 0.049515566048790496 0.03654162132698924 0.025472126494665723 0.016352568480485365 0.009220421504467294 0.004104993088376991 0.0010273036248318428 -1.1905898466964329e-17; 0.0 0.

LoadError: ArgumentError: Provided trajectories must have fixed timesteps

# **Create UnitarySmoothPulseProblem for each quantum system**

In [12]:
# set time parameters
T = 50;
Δt = 0.2

# use the exported gate dictionary to get the gates we need
σx = get_gate(:X)
σy = get_gate(:Y)
σz = get_gate(:Z)

# Target unitary
U_goal = σx

# Set bound on the drive
a_bound = 1.0

# Define options for the optimizer
options = Options(max_iter=30)

# Create UnitarySmoothPulseProblem for each quantum system
problems = [UnitarySmoothPulseProblem(
    sys.H_drift, 
    sys.H_drives, 
    U_goal, 
    T, 
    Δt;
    a_bound=a_bound, 
    options=options,
) for sys in quantum_systems]


applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds o

6-element Vector{QuantumControlProblem}:
 QuantumControlProblem(Ipopt.Optimizer, MathOptInterface.VariableIndex[MOI.VariableIndex(1) MOI.VariableIndex(16) … MOI.VariableIndex(721) MOI.VariableIndex(736); MOI.VariableIndex(2) MOI.VariableIndex(17) … MOI.VariableIndex(722) MOI.VariableIndex(737); … ; MOI.VariableIndex(14) MOI.VariableIndex(29) … MOI.VariableIndex(734) MOI.VariableIndex(749); MOI.VariableIndex(15) MOI.VariableIndex(30) … MOI.VariableIndex(735) MOI.VariableIndex(750)], QuantumSystem(sparse([1, 2], [1, 2], ComplexF64[0.0038369096529725535 + 0.0im, -0.0038369096529725535 + 0.0im], 2, 2), SparseArrays.SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1], [1, 2], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im], 2, 2), sparse([2, 1], [1, 2], ComplexF64[0.0 + 1.0im, 0.0 - 1.0im], 2, 2)], sparse([3, 4, 1, 2], [1, 2, 3, 4], [-0.0038369096529725535, 0.0038369096529725535, 0.0038369096529725535, -0.0038369096529725535], 4, 4), SparseArrays.SparseMatrixCSC{Float64, Int64}[sparse([4, 3, 2, 1], [1, 2

# **Create a direct sum problem**

In [8]:
# Define IPOPT options
ops = Options(print_level=1)

# Create individual UnitarySmoothPulseProblems
problems = [UnitarySmoothPulseProblem(system, U_goal, T, Δt, free_time=false, ipopt_options=ops) for system in quantum_systems]

# Define a final fidelity
final_fidelity = 0.99

# Create a direct sum problem
direct_sum_prob = UnitaryDirectSumProblem(problems, final_fidelity, ipopt_options=ops) # maybe not the correct way

applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: initial value of Ũ⃗
applying constraint: ini

QuantumControlProblem(Ipopt.Optimizer, MathOptInterface.VariableIndex[MOI.VariableIndex(1) MOI.VariableIndex(85) … MOI.VariableIndex(4033) MOI.VariableIndex(4117); MOI.VariableIndex(2) MOI.VariableIndex(86) … MOI.VariableIndex(4034) MOI.VariableIndex(4118); … ; MOI.VariableIndex(83) MOI.VariableIndex(167) … MOI.VariableIndex(4115) MOI.VariableIndex(4199); MOI.VariableIndex(84) MOI.VariableIndex(168) … MOI.VariableIndex(4116) MOI.VariableIndex(4200)], QuantumSystem(sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ComplexF64[0.0038369096529725535 + 0.0im, -0.0038369096529725535 + 0.0im, -0.0014119729372972126 - 0.0im, 0.0014119729372972126 - 0.0im, 0.010836457824888781 + 0.0im, -0.010836457824888781 + 0.0im, 0.0003908681897150272 + 0.0im, -0.0003908681897150272 + 0.0im, 0.0012422732333752225 + 0.0im, -0.0012422732333752225 + 0.0im, -0.01898018179596325 - 0.0im, 0.01898018179596325 - 0.0im], 12, 12), SparseArrays.SparseMatrixCSC{ComplexF64, Int64}[s

In [9]:
# Solve the direct_sum problem
solve!(direct_sum_prob; max_iter=30) #Guidance need as to why it's not showing any output. I think doing something strange here!